In [1]:
import re, os, sys, json, time, requests
from pathlib import Path
import pandas as pd
import numpy as np

# TODO just load the parquets and run parse_eval
# or run my own eval

In [2]:
# TODO load config.json, load eval.parquet
fs = sorted(Path('../outputs').glob('**/eval.parquet'))
fs;

In [3]:
def load_config(f):
    # print(f.parent.parent.stem)
    config = json.load((f.parent / 'config.json').open())
    log = (f.parent / 'log.txt').open().read()

    # dataframe of all non list/dict/tuple in config
    config_df = pd.DataFrame({k: v for k, v in config.items() if not isinstance(v, (list, dict, tuple))}, index=[0])
    config_df['log'] = log
    config_df['file'] = f
    config_df['ts'] = f.parent.stem
    try:
        _, method, _ = f.parent.parent.stem.split('_', 2) # this is {model}_{method}_{dataset}
    except ValueError as e:
        print(e)
        print(f"cannot split `{f.parent.parent.stem}` from {f} like [f.parent.parent.stem.split('_', 3)]")
        method = ''
    config_df['method'] = method

    # put key cols first
    key_columns = ['dataset', 'base_model']
    columns = key_columns + [c for c in config_df.columns if not c in key_columns]
    return config_df[columns]

configs = [load_config(f) for f in fs]
df_configs = pd.concat(configs)
df_configs



/tmp/ipykernel_1238407/3622691232.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_configs = pd.concat(configs)


,dataset,base_model,verbose,dev,load_in_4bit,load_in_8bit,use_gradient_checkpointing,batch_size,n_samples,eval_samples,...,ts,method,β,reverse_pref,scale_orth,neg_slope,mag_clip,weight_dim,collect_input,collect_hs
0,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,1,False,False,False,False,16,5400,None,...,20241013042626,dpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,1,False,False,False,False,16,5400,None,...,20241013103646,projbp,0.2,False,False,0.8,0.6,NaN,NaN,NaN
0,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,1,False,False,False,False,16,5400,None,...,20241013044358,projgrad,0.8,True,False,0.0,NaN,1.0,NaN,NaN
0,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,1,False,False,False,False,16,5400,None,...,20241013055756,sideETHERMSE,NaN,NaN,NaN,NaN,NaN,NaN,True,True
0,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,1,False,False,False,False,16,5400,None,...,20241013040725,sideETHERPrefVec,NaN,NaN,NaN,NaN,NaN,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,us_history_textbook,wassname/llama-3-2-1b-sft,0,False,False,False,False,16,3600,128,...,2024-10-05_05-35-47,side-SVD-MSE,NaN,NaN,NaN,NaN,NaN,NaN,False,False
0,us_history_textbook,wassname/llama-3-2-1b-sft,1,False,False,False,False,16,3600,None,...,2024-10-11_00-09-35,side-SVD-MSE,NaN,NaN,NaN,NaN,NaN,NaN,True,True
0,us_history_textbook,wassname/llama-3-2-1b-sft,2,True,True,False,False,2,64,750,...,2024-10-11_04-36-06,side-SVD-MSE,NaN,NaN,NaN,NaN,NaN,NaN,True,True
0,us_history_textbook,wassname/llama-3-2-1b-sft,1,False,False,False,False,16,3600,None,...,2024-10-11_00-05-32,side-SVD-PrefVec,NaN,NaN,NaN,NaN,NaN,NaN,True,True


In [4]:
# df_evals = [pd.read_parquet(f) for f in fs]
# df_eval = pd.concat(df_evals).reset_index()
# df_eval
from reprpo.training import parse_eval, key_metrics
from tqdm.auto import tqdm

data_acc = []
data_rel_acc = []
data_rel_ppl = []
data_rel_pref = []

for i in tqdm(range(len(fs))):

    df_res2= pd.read_parquet(fs[i])
    config = df_configs.iloc[i]

    ds_alias = dict(list(zip(["train", "test", "oos", "rnd"], df_res2['dataset'].unique())))
    # assert ds_alias['train']==config['dataset']
    assert 'train' in ds_alias['train']
    adapter_name = df_res2[["adapter"]].query('adapter!="base"').values[0, 0]
    df_acc = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
        .mean()
        .unstack()
        .T
    )
    ds_alias_rev = {v:k for k, v in ds_alias.items()}
    df_acc = df_acc.rename(columns=ds_alias_rev)
    
    df_rel = key_metrics(df_res2, adapter_name, ds_alias)
    
    df_acc = df_acc[['train', 'test', 'oos', 'rnd']]

    data_acc.append(dict(
        base_model=config['base_model'],
        train_dataset=config['dataset'],
        method=adapter_name,
        **df_acc.loc[adapter_name].to_dict()
    ))
    data_acc.append(dict(
        base_model=config['base_model'],
        train_dataset=config['dataset'],
        method='base',
        **df_acc.loc['base'].to_dict()
    ))

    data_rel_acc.append(dict(
        base_model=config['base_model'],
        train_dataset=config['dataset'],
        method=adapter_name,
        **df_rel.iloc[0].to_dict()
    ))
    data_rel_ppl.append(dict(
        base_model=config['base_model'],
        train_dataset=config['dataset'],
        method=adapter_name,
        **df_rel.iloc[1].to_dict()
    ))
    data_rel_pref.append(dict(
        base_model=config['base_model'],
        train_dataset=config['dataset'],
        method=adapter_name,
        **df_rel.iloc[2].to_dict()
     ))


  0%|          | 0/2367 [00:00<?, ?it/s]

In [5]:
import seaborn as sns
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
def style_df(df, caption=''):
    return (df.style
        .background_gradient(cmap, axis=1)
        # .set_caption(caption)
        .format(precision=3)
        .highlight_max(axis=0, props='font-weight:bold;')
    )


In [6]:
metrics = dict(
    acc=data_acc, 
    rel_acc=data_rel_acc, 
    rel_ppl=data_rel_ppl,
    rel_pref=data_rel_pref,
)

for metric, data in metrics.items():
    print(f'# {metric}:\n')
    for (base_model,train_dataset),df in pd.DataFrame(data).groupby(['base_model', 'train_dataset']):
        print(f'## {base_model} - {train_dataset}')
        df = df.drop(['base_model','train_dataset'], axis='columns')
        df = df.groupby('method', as_index=True).mean().sort_values('oos', ascending=False).dropna(axis=1)
        if (
            (len(df)>2) and 
            (df.std()>0).all()
        ):
                display(style_df(df, f'{metric}: {base_model} {train_dataset}'))
        else:
             print(f'skipped {df.shape}')
            

    print('\n')

# acc:

## princeton-nlp/Llama-3-Base-8B-SFT - alpaca_easy


,train,test,oos,rnd
method,,,,
side-None-PrefVec,0.985,0.955,0.769,0.632
side-ETHER-PrefVec,0.985,0.950,0.767,0.653
projgrad,0.999,0.982,0.759,0.679
base,0.973,0.956,0.741,0.677
dpo,0.999,0.978,0.739,0.649


## princeton-nlp/Llama-3-Base-8B-SFT - alpaca_low_quality


,train,test,oos,rnd
method,,,,
projgrad,0.999,0.991,0.111,0.705
side-ETHER-PrefVec,0.912,0.920,0.111,0.673
side-None-PrefVec,0.903,0.919,0.109,0.684
base,0.829,0.863,0.108,0.677
dpo,0.997,0.992,0.105,0.684


## princeton-nlp/Llama-3-Base-8B-SFT - alpaca_mmlu


...

,train,test,oos,rnd
method,,,,
dpo,132.097,84.356,54.608,22.818
projgrad,118.423,73.527,48.958,22.649
side-ETHER-PrefVec,13.781,9.398,7.732,2.376
side-None-PrefVec,12.287,7.499,6.974,0.887


## princeton-nlp/Llama-3-Base-8B-SFT - alpaca_short
skipped (1, 4)
## princeton-nlp/Llama-3-Base-8B-SFT - us_history_textbook


,train,test,oos,rnd
method,,,,
projgrad,419.315,400.767,207.197,6.557
dpo,370.233,353.863,193.468,7.550
side-SVD-Rank,75.595,68.656,48.926,0.432
side-Ortho-Rank,80.533,75.384,48.461,0.876
side-None-Rank,82.202,77.220,40.024,0.481
side-None-PrefVec,28.400,23.872,38.523,0.273
side-HRA-PrefVec,24.369,20.753,33.580,-0.066
side-ETHER-Rank,57.042,52.891,32.688,0.695
side-Ortho-PrefVec,23.436,20.063,31.756,-0.045


## reciprocate/tiny-llama - us_history_textbook
skipped (2, 4)
## wassname/llama-3-2-1b-sft - us_history_textbook


,train,test,oos,rnd
method,,,,
dpo,325.317,346.639,159.940,0.071
projgrad,299.714,320.749,139.283,-0.062
side-Ortho-Rank,70.039,62.565,30.594,-0.148
side-ETHER-Rank,94.977,81.113,28.400,0.059
side-None-Rank,94.977,81.113,28.400,0.059
side-SVD-Rank,94.977,81.113,28.400,0.059
side-HRA-PrefVec,18.882,17.279,23.988,-0.073
side-None-PrefVec,18.882,17.279,23.988,-0.073
side-SVD-PrefVec,18.882,17.279,23.988,-0.073
